Preparing to submit wold stranded samples....

The spreasheet has a bunch of library accessions in the files accession column which is wrong. I think it needs to be fixed.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [4]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-first79_hff-posted-files.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [5]:
award = 'UM1HG009443'

# Register Biosamples

In [6]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)


# Register Libraries

In [7]:
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)


In [13]:
library = server.get_json('barbara-wold:23102')
print(library['aliases'], library.get('average_fragment_size'), library.get('fragment_length_CV'))

['barbara-wold:23102', 'barbara-wold:ENC4_cDNA438'] 327 29


In [20]:
for i, row in libraries.iterrows():
    library = server.get_json(row['accession'])
    if library.get('average_fragment_size') is None and library.get('fragment_length_CV') is None:
        update = {
            'average_fragment_size': int(row['average_fragment_size:integer']),
            'fragment_length_CV': float(row['fragment_length_CV:number'])
        }
        print(library['aliases'], library.get('average_fragment_size'), library.get('fragment_length_CV'), update)
        server.patch_json(library['@id'], update)
        

['barbara-wold:22161'] None None {'average_fragment_size': 247, 'fragment_length_CV': 71.8}
['barbara-wold:22162'] None None {'average_fragment_size': 243, 'fragment_length_CV': 42.1}


# Register Experiments

In [9]:
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)


# Register Replicates

In [10]:
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)


# Check Files

In [11]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
